<p style="font-weight:bold;"> <span style="font-size: 36px"> CHashFlower calculation Test </span> </p>

In [0]:
#!import "../Initialization/InitSystemorphToMemory"

# Args

In [0]:
var reportingNode = "DE";
var year = 2022;
var month = 12;
var importFormat = ImportFormats.Cashflow;
var periodicity = Periodicity.Monthly;
var scenario = (string)null;

ImportArgs args = new ImportArgs(reportingNode, year, month, periodicity, scenario, importFormat);

# Basic

In [0]:
var workspace = Workspace.CreateNew();
workspace.Initialize(x => x.FromSource(DataSource));

In [0]:
var basicCashflows = @"@@BasicCashflow
DataNode,AmountType,Shape,InitialValue,Length,Shift,Values0,Values1,Values2,Values3,Values4,
G.BBA.LRC.2020.P,PR,Constant,10,5,0,,,,,
G.BBA.LRC.2020.P,PR,Linear,10,5,1,2,-1,0.5,0.2
";

In [0]:
await Import.FromString(basicCashflows).WithType<BasicCashflow>().WithTarget(workspace).ExecuteAsync()

In [0]:
var storage = new CashflowStorage(args, DataSource, workspace);
await storage.InitializeAsync();
var identities = storage.Identities;

public interface IModel : IMutableScopeWithStorage<CashflowStorage>{}
var universe = Scopes.ForStorage(storage).ToScope<IModel>();

In [0]:
var cashflowVariables = Scopes.ForIdentities(identities, storage).ToScopes<Compute>().SelectMany(x => x.FpaVariables).ToArray();
cashflowVariables

In [0]:
double[] array1 = new double[] { 2.0, 3.0, 6.7 };
array1.SequenceEqual(new double[] { 2.0, 3.0, 6.7 }, Precision);

In [0]:
cashflowVariables.Single(x => x.Layer.Contains("Constant")).Values.SequenceEqual(new double[] { 10, 10, 10, 10, 10 }, Precision).Should().BeTrue();

In [0]:
cashflowVariables.Single(x => x.Layer.Contains("Linear")).Values.SequenceEqual(new double[] { 0, 20, -20, -10, -2}, Precision).Should().BeTrue();

// Check computed : new double[] { 0, 10, -10, -5, -1} ; 
// The first weight is used for the skipped value => useless Values input up to Shift
// The first value after the skip is the InitialValue making the Value1 column useless

In [0]:
workspace.Dispose()

# Bounded

In [0]:
var workspace = Workspace.CreateNew();
workspace.Initialize(x => x.FromSource(DataSource));

In [0]:
var boundedCashflows = @"@@BoundedCashflow  
DataNode,AmountType,Shape,InitialValue,Length,Shift,Minimum,Maximum
G.BBA.LRC.2020.P,PR,Random,6000,4,1,100,101
";

In [0]:
await Import.FromString(boundedCashflows).WithType<BoundedCashflow>().WithTarget(workspace).ExecuteAsync()

In [0]:
var storage = new CashflowStorage(args, DataSource, workspace);
await storage.InitializeAsync();
var identities = storage.Identities;

public interface IModel : IMutableScopeWithStorage<CashflowStorage>{}
var universe = Scopes.ForStorage(storage).ToScope<IModel>();

In [0]:
var cashflowVariables = Scopes.ForIdentities(identities, storage).ToScopes<Compute>().SelectMany(x => x.FpaVariables).ToArray();
cashflowVariables

In [0]:
cashflowVariables.Single().Values.First().Should().BeApproximately(0d, Precision);
cashflowVariables.Single().Values.Skip(1).All(x => x < 101d && x > 100d).Should().BeTrue();

In [0]:
workspace.Dispose()

# Reference